In [37]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from car_dynamics.envs import DynamicParams, DynamicBicycleModel

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
#TODO: load data
df = pd.read_csv('/Users/wenlixiao/Dropbox/School/Graduate/LeCAR/safe-learning-control/playground/offroad/data/vicon-data-clean/vicon-circle-data-20240225-215833.csv')

In [39]:
%matplotlib inline

H = 4

def plot(Ta=8., Tb=1., K_RFY= 1., K_FFY= 1.):
    model_params = DynamicParams(num_envs=1, Ta=Ta, Tb=Tb, K_FFY=K_FFY, K_RFY=K_RFY)
    dynamics = DynamicBicycleModel(model_params, device=DEVICE)

    err = []
    dt_list = []
    for i in range(1, df.shape[0]-1-H):
        ptr = i
        
        dt = df['time'][i] - df['time'][i-1]
        x = df['pos_x'][i]
        y = df['pos_y'][i]
        v_vec = np.array([df['pos_x'][i] - df['pos_x'][i-1], df['pos_y'][i] - df['pos_y'][i-1]]) / dt
        
        vx = v_vec[0] * np.cos(df['yaw'][i]) + v_vec[1] * np.sin(df['yaw'][i])
        vy = v_vec[1] * np.cos(df['yaw'][i]) - v_vec[0] * np.sin(df['yaw'][i])
                
        psi = df['yaw'][i]
        omega = (df['yaw'][i] - df['yaw'][i-1]) 
        omega = np.arctan2(np.sin(omega), np.cos(omega)) / dt
        
        err.append([])
        for h in range(H):
            
            ## calc next
            dt = df['time'][ptr + 1] - df['time'][ptr]
            v_vec_ip1 = np.array([df['pos_x'][ptr+1] - df['pos_x'][ptr], df['pos_y'][ptr+1] - df['pos_y'][ptr]]) / dt
            vx_ip1 = v_vec_ip1[0] * np.cos(df['yaw'][ptr+1]) + v_vec_ip1[1] * np.sin(df['yaw'][ptr+1])
            vy_ip1 = v_vec_ip1[1] * np.cos(df['yaw'][ptr+1]) - v_vec_ip1[0] * np.sin(df['yaw'][ptr+1])
            omega_ip1 = (df['yaw'][ptr+1] - df['yaw'][ptr])
            omega_ip1 = np.arctan2(np.sin(omega_ip1), np.cos(omega_ip1)) / dt
        
            x, y, psi, vx, vy, omega = dynamics.single_step_numpy(
                np.array([ x, y, psi, vx, vy, omega ]),
                np.array([ df['target_vel'][ptr], df['target_steer'][ptr] ])
            )
        
            err[-1].append(
                np.abs(np.array([
                    x - df['pos_x'][ptr+1],
                    y - df['pos_y'][ptr+1],
                    psi - df['yaw'][ptr+1],
                    vx - vx_ip1,
                    vy - vy_ip1,
                    omega - omega_ip1,
                ]))
            )
            ptr += 1
        
    err = np.array(err)
    mean_err = np.mean(err, axis=0)
    # plt.plot(err[:, :, 3])
    plt.plot(mean_err[:, 0])
    # plt.plot(df['target_vel'])
    # plt.plot(dt_list)
    # plt.ylim([0, 5])
    plt.show()
interact(plot, Ta=(0.1, 10, 0.1), Tb=(0.1, 10, 0.1))

interactive(children=(FloatSlider(value=8.0, description='Ta', max=10.0, min=0.1), FloatSlider(value=1.0, desc…

<function __main__.plot(Ta=8.0, Tb=1.0, K_RFY=1.0, K_FFY=1.0)>

In [12]:
df

,date,time,pos_x,pos_y,yaw,target_vel,target_steer
0,2024-02-25 21:58:36.329434,1.708916e+09,1.242302,0.459714,-2.860860,0.1,0.0
1,2024-02-25 21:58:36.379683,1.708916e+09,1.242179,0.460084,-2.861503,0.1,0.0
2,2024-02-25 21:58:36.430068,1.708916e+09,1.242277,0.459803,-2.861013,0.1,0.0
3,2024-02-25 21:58:36.480645,1.708916e+09,1.240277,0.459866,-2.861637,0.1,0.0
4,2024-02-25 21:58:36.530917,1.708916e+09,1.239047,0.459489,-2.861462,0.1,0.0
...,...,...,...,...,...,...,...
2785,2024-02-25 22:00:56.450910,1.708916e+09,2.147191,3.894629,-0.016733,0.0,0.0
2786,2024-02-25 22:00:56.501175,1.708916e+09,2.147151,3.894602,-0.016843,0.0,0.0
2787,2024-02-25 22:00:56.551449,1.708916e+09,2.147204,3.894769,-0.016273,0.0,0.0
2788,2024-02-25 22:00:56.601770,1.708916e+09,2.147194,3.894607,-0.016795,0.0,0.0
